<a href="https://colab.research.google.com/github/Jsgithubchannel/AIFFEL_quest_cr/blob/main/%08DLwithPython/CH2_5_1_%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf

# 단순 Dense 클래스
class NaiveDense:
  def __init__(self, input_size, output_size, activation):
    self.activation = activation

    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    b_shape = (output_size, )
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)

  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self):
    return [self.W, self.b]

In [8]:
# 단순 Sequential 클래스
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers
  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [9]:
# 케라스와 유사한 모델 만들기
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation = tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation = tf.nn.softmax)
])
assert len(model.weights) == 4

In [10]:
# 배치 제너레이터
import math

class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

In [14]:
# 훈련 스텝 실행

## 가중치 업데이트 수동 구현
# learning_rate = 1e-3

# def update_weights(gradients, weights):
#   for g, w in zip(gradients, weights):
#     w.assign_sub(g * learning_rate)

from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
  optimizer.apply_gradients(zip(gradients, weights))

def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
        labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)
  gradients = tape.gradient(average_loss, model.weights) # 가중치에 대한 손실의 그레이디언트 계산
  update_weights(gradients, model.weights)
  return average_loss

In [15]:
# 전체 훈련 루프
def fit(model, images, labels, epochs, batch_size=128):
  for epoch_counter in range(epochs):
    print(f"에포크 {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"{batch_counter}번째 배치 손실: {loss: .2f}")

In [16]:
# 함수 테스트
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
에포크 0
0번째 배치 손실:  6.56
100번째 배치 손실:  2.23
200번째 배치 손실:  2.20
300번째 배치 손실:  2.09
400번째 배치 손실:  2.24
에포크 1
0번째 배치 손실:  1.90
100번째 배치 손실:  1.87
200번째 배치 손실:  1.82
300번째 배치 손실:  1.72
400번째 배치 손실:  1.84
에포크 2
0번째 배치 손실:  1.58
100번째 배치 손실:  1.57
200번째 배치 손실:  1.49
300번째 배치 손실:  1.43
400번째 배치 손실:  1.51
에포크 3
0번째 배치 손실:  1.31
100번째 배치 손실:  1.33
200번째 배치 손실:  1.22
300번째 배치 손실:  1.21
400번째 배치 손실:  1.27
에포크 4
0번째 배치 손실:  1.11
100번째 배치 손실:  1.15
200번째 배치 손실:  1.03
300번째 배치 손실:  1.05
400번째 배치 손실:  1.10
에포크 5
0번째 배치 손실:  0.97
100번째 배치 손실:  1.02
200번째 배치 손실:  0.90
300번째 배치 손실:  0.93
400번째 배치 손실:  0.98
에포크 6
0번째 배치 손실:  0.86
100번째 배치 손실:  0.91
200번째 배치 손실:  0.80
300번째 배치 손실:  0.84
400번째 배치 손실:  0.89
에포크 7
0번째 배치 손실:  0.78
100번째 배치 손실:  0.83
200번째 배치 손실:  0.72
300번째 배치 손실:  0.78
400번째 배치 손실:  0.83
에포크 8
0번째 배치 손실:  0.72
100번째 배치 손실:  0.76
200번째 배치 손실:  0.66
300번째 배치 손실:  0.72
400번째 배치 손실:  0.77
에포크 9
0번째 배치 손실:  0.67
100번째 배치 손실:  0.71
200번째 배치 손실:  0.

In [19]:
# 모델 평가
import numpy as np
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"정확도: {matches.mean(): .2f}")

정확도:  0.82


In [26]:
predictions[0].argmax()

7